In [ ]:
def calcLogFreqSpec(S, f_if, fmin = 32.703, R = 100, numOctaves = 8, winsize = 2048):
    '''
    Calculate the log frequency spectrogram for an input STFT
    
    Arguments:
    S -- input STFT
    f_if -- instantaneous frequency estimations for S
    optional--minimum frequency fmin, cents per bin R, number of octaves of STFT
    
    Returns:
    logfreq -- log frequency matrix
    '''
    numBins = int((int(winsize/2) + 1) * 2)
    bin_start = np.round(1200/R * np.log2(np.clip(fmin, 1*10**-9, np.inf)/440) + 69*100/R).astype(int)
    bin_end = (bin_start + numOctaves*1200/R).astype(int)
    bins = np.clip(np.round(1200/R * np.log2(np.clip(f_if, 1*10**-9, np.inf)/440) + 69*100/R), 0, bin_end+1).astype(int).flatten()
    logfreq = np.zeros((S.shape[0], S.shape[1]-1))
    colMat = np.matlib.repmat(np.arange(S.shape[1]-1), S.shape[0], 1).flatten()
    S = S[:, :-1].flatten()
    for i in range(0, S.shape[0]):
        logfreq[bins[i]][colMat[i]] += abs(S[i])**2
    logfreq = 2*logfreq[int(bin_start):int(bin_start+numOctaves*12*100/R), :]
    return logfreq

In [ ]:
def principalArgF(theta):
    '''
    Calculate principle argument function of an input angle theta
    
    Arguments:
    theta -- input angle
    
    Returns:
    theta but between -pi and pi
    '''
    return  np.mod((theta+np.pi), 2*np.pi) - np.pi

In [ ]:
def estimateIF(S, sr = 22050, hop = 512):
    '''
    Calculate the instantaneous frequency estimation for an STFT
    
    Arguments:
    S -- STFT
    
    Returns:
    Inst -- instantaneous frequency estimation
    '''
    phases = np.angle(S[:, 1:]) - np.angle(S[:, 0:-1])
    w = (np.broadcast_to(np.pi*sr/S.shape[0]*np.arange(S.shape[0]), (S.shape[1]-1, S.shape[0])).transpose())
    Inst = (w + principalArgF((phases - w*hop))/hop)/(2*np.pi)
    return Inst